# W&B Image Normalization Demo

This notebook demonstrates how `wandb.Image` automatically normalizes different types of image data and how to control this behavior.

## What you'll learn:
- How `wandb.Image` normalizes PyTorch tensors and NumPy arrays
- When normalization is applied vs when it's not
- How to avoid unwanted normalization
- Best practices for image logging

## Setup

First, let's install the required dependencies and import the necessary libraries.

In [ ]:
# Install required packages
!pip install wandb torch torchvision pillow matplotlib numpy

In [ ]:
import wandb
import torch
import numpy as np
from PIL import Image as PILImage
import matplotlib.pyplot as plt

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## Initialize W&B

Let's start a W&B run to log our examples.

In [ ]:
# Initialize W&B run
run = wandb.init(
    project="image-normalization-demo",
    name="normalization-examples",
    config={
        "description": "Demonstrating wandb.Image normalization behavior"
    }
)

## Understanding Image Normalization

When you pass PyTorch tensors or NumPy arrays to `wandb.Image`, the pixel values are automatically normalized to the range [0, 255] unless you set `normalize=False`.

**Normalization is applied to:**
- PyTorch tensors (format: `(channel, height, width)`)
- NumPy arrays (format: `(height, width, channel)`)

**Normalization is NOT applied to:**
- PIL Images (passed as-is)
- File paths (loaded as-is)

**Normalization algorithm:**
- [0, 1] range: values are multiplied by 255
- [-1, 1] range: values are rescaled using `255 * 0.5 * (data + 1)`
- Other ranges: values are clipped to [0, 255]

## Example 1: [0, 1] Range Data

When your tensor/array values are in the [0, 1] range, `wandb.Image` will multiply all values by 255.
This example creates a 64x64 pixel image with three color channels (RGB) and random values for each pixel between 0 and 1. It then converts the image from a NumPy array to a PyTorch tensor, changing the format from (height, width, channels) to (channels, height, width) which is what PyTorch expects.

The `wandb.Image(tensor_0_1)` function automatically:
1. **Detects** that your values are in the [0, 1] range
2. **Multiplies every value by 255** to convert to [0, 255] range
3. **Converts to uint8** (8-bit integers, which is standard for images)

This ensures your image displays with the correct brightness and colors, since most image viewers expect values in the [0, 255] range.

In [ ]:
# Create test data in [0, 1] range
data_0_1 = np.random.rand(64, 64, 3)
print(f"Original data range: [{data_0_1.min():.3f}, {data_0_1.max():.3f}]")

# Convert to PyTorch tensor (channel, height, width format)
tensor_0_1 = torch.from_numpy(data_0_1).permute(2, 0, 1).float()
print(f"Tensor shape: {tensor_0_1.shape}")
print(f"Tensor range: [{tensor_0_1.min():.3f}, {tensor_0_1.max():.3f}]")

# Visualize the original data
plt.figure(figsize=(8, 6))
plt.imshow(data_0_1)
plt.title(f'[0, 1] Range Data\nValues will be multiplied by 255')
plt.colorbar()
plt.axis('off')
plt.show()

# Log to W&B
wandb.log({
    "example_0_1_range": wandb.Image(
        tensor_0_1,
        caption="[0, 1] range tensor - values will be multiplied by 255"
    )
})

## Example 2: [-1, 1] Range Data

When your tensor/array values are in the [-1, 1] range, `wandb.Image` will rescale them using the formula: `255 * 0.5 * (data + 1)`
This example creates a 64x64 pixel image with three color channels and random values for each pixel between -1 and 1. This range is common in machine learning when using normalized data or when working with models that output values in this range.

The `wandb.Image(tensor_neg1_1)` function automatically:
1. **Detects** that your values are in the [-1, 1] range
2. **Rescales using the formula**: `255 * 0.5 * (data + 1)`
3. **Converts to uint8** for proper image display

This rescaling maps:
- -1 → 0 (black)
- 0 → 127.5 (gray)
- 1 → 255 (white)

This ensures that negative values are properly handled and your image displays correctly.

**Note on visual contrast**: The [-1, 1] normalization creates higher visual contrast compared to [0, 1] normalization. This is because:
- Negative values (like -0.8) become very dark (around 25)
- Positive values (like 0.8) become very bright (around 230)
- Values near 0 become mid-gray (127.5)

This "stretches" the visual range, making differences between pixel values more pronounced. If you want less contrast, you might consider preprocessing your data to a [0, 1] range before logging.

In [ ]:
# Create test data in [-1, 1] range
data_neg1_1 = np.random.rand(64, 64, 3) * 2 - 1
print(f"Original data range: [{data_neg1_1.min():.3f}, {data_neg1_1.max():.3f}]")

# Convert to PyTorch tensor
tensor_neg1_1 = torch.from_numpy(data_neg1_1).permute(2, 0, 1).float()
print(f"Tensor shape: {tensor_neg1_1.shape}")
print(f"Tensor range: [{tensor_neg1_1.min():.3f}, {tensor_neg1_1.max():.3f}]")

# Visualize the original data
plt.figure(figsize=(8, 6))
plt.imshow(data_neg1_1, cmap='RdBu_r')
plt.title(f'[-1, 1] Range Data\nValues will be rescaled: -1→0, 0→127.5, 1→255')
plt.colorbar()
plt.axis('off')
plt.show()

# Log to W&B
wandb.log({
    "example_neg1_1_range": wandb.Image(
        tensor_neg1_1,
        caption="[-1, 1] range tensor - values will be rescaled"
    )
})

## Example 3: Avoiding Normalization with PIL Images

To avoid normalization, you can convert your tensors to PIL Images before passing them to `wandb.Image`.
This example shows how to prevent automatic normalization by converting your PyTorch tensor to a PIL Image first. This is useful when you want to control exactly how your pixel values are processed.

The process involves:
1. **Creating a tensor** with values in [0, 1] range
2. **Converting to NumPy array** and permuting dimensions back to (height, width, channels)
3. **Multiplying by 255** manually to convert to [0, 255] range
4. **Converting to uint8** for proper image format
5. **Creating a PIL Image** from the processed array

When you pass a PIL Image to `wandb.Image`, it is passed through without any normalization, giving you complete control over the pixel values.

**When to use PIL conversion vs normalize=False:**

**Use PIL conversion when:**
- You want complete control over pixel values
- You need custom preprocessing (filters, brightness adjustments, etc.)
- You want to use PIL's image processing capabilities
- You're debugging and want to see exact values being logged

**Use normalize=False when:**
- You want to see raw tensor values as they are
- Your data is already in the correct range (like [0, 255] integers)
- You're debugging normalization issues
- Quick testing without additional processing steps

In [ ]:
# Create tensor with values in [0, 1] range
tensor_0_1 = torch.rand(3, 64, 64)
print(f"Tensor range: [{tensor_0_1.min():.3f}, {tensor_0_1.max():.3f}]")

# Convert to PIL Image to avoid normalization
pil_image = PILImage.fromarray(
    (tensor_0_1.permute(1, 2, 0).numpy() * 255).astype('uint8')
)
print(f"PIL Image size: {pil_image.size}")
print(f"PIL Image mode: {pil_image.mode}")

# Visualize the PIL image
plt.figure(figsize=(8, 6))
plt.imshow(pil_image)
plt.title('PIL Image - No normalization applied')
plt.axis('off')
plt.show()

# Log to W&B
wandb.log({
    "example_pil_no_normalization": wandb.Image(
        pil_image,
        caption="PIL Image - no normalization applied"
    )
})

## Example 4: Using normalize=False

You can also disable normalization by setting `normalize=False`. Values will be clipped to [0, 255].
This example demonstrates how to disable automatic normalization using the `normalize=False` parameter. This is useful for debugging or when you want to see the raw values of your tensor.

When `normalize=False` is set:
1. **No multiplication by 255** occurs
2. **Values are clipped** to the [0, 255] range (values below 0 become 0, values above 255 become 255)
3. **Values are converted to uint8** for image display

This means that if your tensor has values in [0, 1] range, they will be treated as if they were already in [0, 255] range, which will make your image appear very dark since 0.5 becomes 0.5 out of 255 (almost black).

In [ ]:
# Create tensor with values in [0, 1] range
tensor_0_1 = torch.rand(3, 64, 64)
print(f"Tensor range: [{tensor_0_1.min():.3f}, {tensor_0_1.max():.3f}]")

# Disable normalization
wandb.log({
    "example_normalize_false": wandb.Image(
        tensor_0_1,
        normalize=False,
        caption="Normalization disabled - values will be clipped to [0, 255]"
    )
})

# Also log with normal normalization for comparison
wandb.log({
    "example_normalize_true": wandb.Image(
        tensor_0_1,
        normalize=True,
        caption="Normalization enabled - values will be multiplied by 255"
    )
})

print("Logged both normalized and non-normalized versions for comparison")

## Best Practices

Based on what we've learned, here are some best practices for working with `wandb.Image`:

### 1. **For consistent results**: Pre-process your data to the expected [0, 255] range before logging
### 2. **To avoid normalization**: Convert tensors to PIL Images using `PILImage.fromarray()`
### 3. **For debugging**: Use `normalize=False` to see the raw values (they will be clipped to [0, 255])
### 4. **For precise control**: Use PIL Images when you need exact pixel values

### Common Issues to Watch Out For:
- **Unexpected brightness**: If your tensor values are in [0, 1] range, they will be multiplied by 255, making the image much brighter
- **Data loss**: Values outside the [0, 255] range will be clipped, potentially losing information
- **Inconsistent behavior**: Different input types (tensor vs PIL vs file path) may produce different results

In [ ]:
# Finish the W&B run
wandb.finish()
print("✅ Demo completed! Check your W&B dashboard to see all the logged images.")